# Using Numpy only

## Linear Regression

In [1]:
import numpy as np

class LinearRegression():
    def __init__(self, eps=1e-6):
        self.eps = eps

    def fit(self, x, y, lr=0.01, add_intercept=True):
        if add_intercept:
            x = np.hstack([np.ones((x.shape[0], 1)), x])
        m, n = x.shape

        self.theta = np.zeros(n)

        while True:
            theta_old = self.theta.copy()
            h_x = x.dot(self.theta)
            grad = (x.T.dot(h_x - y)) / m
            self.theta -= lr * grad

            if np.linalg.norm(self.theta - theta_old, ord=1) < self.eps:
                break

    def predict(self, x, add_intercept=True):
        if add_intercept:
            x = np.hstack([np.ones((x.shape[0], 1)), x])
        return x.dot(self.theta)

## Logistic Regression

In [2]:
import numpy as np

class LogisticRegression():
    def __init__(self, eps=1e-6):
        self.eps = eps

    def fit(self, x, y, add_intercept=True, lr=0.01, max_iter=10000):
        if add_intercept:
            x = np.hstack([np.ones((x.shape[0], 1)), x])

        m, n = x.shape
        self.theta = np.zeros(n)

        for _ in range(max_iter):
            theta_old = self.theta.copy()
            z = np.clip(x.dot(self.theta), -500, 500)
            h_x = 1 / (1 + np.exp(-z))
            grad = (x.T.dot(h_x - y)) / m
            self.theta -= lr * grad

            if np.linalg.norm(self.theta - theta_old, ord=1) < self.eps:
                break

    def predict_proba(self, x, add_intercept=True):
        if add_intercept:
            x = np.hstack([np.ones((x.shape[0], 1)), x])
        z = np.clip(x.dot(self.theta), -500, 500)
        return 1 / (1 + np.exp(-z))

    def predict(self, x, add_intercept=True):
        return (self.predict_proba(x, add_intercept) >= 0.5).astype(int)


# Using Pytorch

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

## Linear regression

In [4]:
torch.manual_seed(0)

x = torch.linspace(0, 10, 100).view(-1, 1)  
y = 2 * x + 3 + torch.randn_like(x) 

x_numpy = x.numpy()
y_numpy = y.numpy()

In [5]:
class LinearRegression_pytorch(nn.Module):
    def __init__(self):
        super(LinearRegression_pytorch, self).__init__()
        self.linear = nn.Linear(1,1)
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegression_pytorch()
criterion = nn.MSELoss()
optimiser = optim.SGD(model.parameters(), lr=0.01)

epochs = 1000
losses = []

for epoch in range(epochs):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    losses.append(loss.item())
    if epoch%100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 246.7126
Epoch [101/1000], Loss: 1.6820
Epoch [201/1000], Loss: 1.2833
Epoch [301/1000], Loss: 1.1359
Epoch [401/1000], Loss: 1.0814
Epoch [501/1000], Loss: 1.0613
Epoch [601/1000], Loss: 1.0538
Epoch [701/1000], Loss: 1.0511
Epoch [801/1000], Loss: 1.0501
Epoch [901/1000], Loss: 1.0497


## Logistic regression

In [ ]:
class LogisticRegression_pytorch(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression_pytorch, self).__init__()
        self.linear = nn.Linear(input,input_dim)
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegression_pytorch(input_dim=x.shape[1])
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

## now use training loop
